In [281]:
import openpyxl
import pandas as pd
import numpy as np

# Load the Excel file
workbook = openpyxl.load_workbook("Temp - VBA.xlsm")

# Read the "Hi" worksheet
hi_sheet = workbook["Hi"]
company_col = pd.DataFrame([cell.value for cell in hi_sheet["A"]], columns=["Company"]).dropna()
time_col = pd.DataFrame([cell.value for cell in hi_sheet["N"]], columns=["TimeSet"]).dropna()

print(company_col)
print(time_col)

  Company
0     AAA
1     BBB
2     CCC
      TimeSet
0  2024-06-20
1  2024-07-20
2  2024-08-20
3  2024-09-20
4  2024-10-20
5  2024-11-20
6  2024-12-20
7  2025-01-20
8  2025-02-20
9  2025-03-20
10 2025-04-20
11 2025-05-20


In [282]:
# Read the "Data" worksheet
data_sheet = workbook["Data"]
data2_sheet = workbook["Data2"]

# Initialize the benchmark dictionary
benchmark = {}

# Iterate over the rows in the benchmark data range
for row in data2_sheet.iter_rows(values_only=True):
    item_name = row[0]
    test_b_value = row[1]
    test_c_value = row[2]
    benchmark[item_name] = (test_b_value, test_c_value)

if "TestA" in benchmark:
    del benchmark["TestA"]

benchmark

{'apple': (10000, 10000),
 'banana': (10000, 20000),
 'gold': (10000, 30000),
 'king': (10000, 2000),
 'kite': (10000, 800),
 'yellow': (10000, 500),
 'green': (10000, 6000),
 'purple': (10000, 7000),
 'queen': (10000, 4000)}

In [283]:
# Create initial empty dataframes

def clear_charts():
    global more_item, less_item, more_item_value, less_item_value
    column_names = time_col["TimeSet"].tolist()
    more_item = pd.DataFrame(columns=["ItemName"] + column_names)
    less_item = pd.DataFrame(columns=["ItemName"] + column_names)
    more_item_value = pd.DataFrame(columns=["ItemName"] + column_names)
    less_item_value = pd.DataFrame(columns=["ItemName"] + column_names)

clear_charts()

In [284]:
# Create a list to store the data
data = []

# Iterate through the rows in the "Data" worksheet
for row in data_sheet.iter_rows(values_only=True):
    data.append(row)

# Create the DataFrame
data_df = pd.DataFrame(data)

# Set the column names (if available)
if data_sheet.max_row > 0:
    data_df.columns = data_sheet[1]

# Display the DataFrame
data_df.columns = range(data_df.shape[1])
data_df

,0,1,2,3,4,5,6,7,8,9,...,69,70,71,72,73,74,75,76,77,78
0,TestA,TestB,TestC,AAA,None,TestA,TestB,TestC,AAA,None,...,None,TestA,TestB,TestC,AAA,None,TestA,TestB,TestC,AAA
1,apple,500,10000,2024-06-20 00:00:00,None,yellow,500,10000,2024-07-20 00:00:00,None,...,None,green,500,10000,2024-08-20 00:00:00,None,green,500,10000,2024-09-20 00:00:00
2,banana,600,20000,None,None,red,600,20000,None,None,...,None,apple,600,20000,None,None,apple,600,20000,None
3,orange,100,30000,None,None,blue,100,30000,None,None,...,None,red,100,30000,None,None,gold,100,30000,None
4,lemon,700,20000,None,None,apple,700,20000,None,None,...,None,blue,700,20000,None,None,queen,700,20000,None
5,red,900,10000,None,None,orange,900,10000,None,None,...,None,orange,900,10000,None,None,red,900,10000,None
6,blue,10,500,None,None,None,None,None,None,None,...,None,kite,10,500,None,None,yellow,10,500,None
7,yellow,500,6000,None,None,None,None,None,None,None,...,None,queen,500,6000,None,None,banana,500,6000,None
8,green,700,7000,None,None,None,None,None,None,None,...,None,gold,700,7000,None,None,blue,700,7000,None
9,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [285]:
# Get processing company and time, delete extra columns, rename
def get_block(data_df, row_extract_start, row_extract_end, col_extract_start, col_extract_end):
    block_df = data_df.iloc[row_extract_start:row_extract_end, col_extract_start:col_extract_end]
    processing_company = block_df.iloc[0, 3]
    processing_time = block_df.iloc[1, 3]
    block_df = block_df.drop(block_df.columns[3], axis=1)
    block_df = block_df.dropna(how='all', axis=0)
    block_df.columns = block_df.iloc[0]
    block_df = block_df.iloc[1:].reset_index(drop=True)
    return block_df, processing_company, processing_time

# Check if the item is in the more_item list already. Add if not.
def check_or_add_item(item_name):
    global more_item, less_item, more_item_value, less_item_value
    if not more_item.empty:
        if item_name in more_item["ItemName"].values:
            return
    new_row = pd.DataFrame({"ItemName": [item_name]})
    more_item = pd.concat([more_item, new_row]).fillna(0)
    less_item = pd.concat([less_item, new_row]).fillna(0)
    more_item_value = pd.concat([more_item_value, new_row]).fillna(0)
    less_item_value = pd.concat([less_item_value, new_row]).fillna(0)

In [286]:
# Clear previous records
clear_charts()

# Slice the data_df slowly
row_extract_start = 0
row_extract_end = 10
col_extract_start = 0
col_extract_end = 4

while row_extract_end <= len(data_df)+5:

    while col_extract_end <= len(data_df.columns):
        block_df, processing_company, processing_time = get_block(data_df, row_extract_start, row_extract_end, col_extract_start, col_extract_end)
        diff = 0

        for _, row in block_df.iterrows():

            # Check item added in four charts or not; also for dictionary
            check_or_add_item(row["TestA"])
            if row["TestA"] not in benchmark:
                benchmark.setdefault(row["TestA"], (0, 0))

           # Compare with benchmark
            if row["TestA"] not in benchmark:
                diff = row["TestB"]
            else:
                diff = row["TestB"] - benchmark[row["TestA"]][0]

            # Update the dataframes
            if diff > 0:
                more_item.loc[more_item["ItemName"] == row["TestA"], processing_time] += 1
                more_item_value.loc[more_item_value["ItemName"] == row["TestA"], processing_time] += row["TestC"] - benchmark[row["TestA"]][1]
            elif diff < 0:
                less_item.loc[less_item["ItemName"] == row["TestA"], processing_time] += 1
                less_item_value.loc[less_item_value["ItemName"] == row["TestA"], processing_time] += benchmark[row["TestA"]][1] - row["TestC"]
            else:
                pass

        col_extract_start += 5
        col_extract_end += 5

        print(processing_company, processing_time)

    row_extract_start += 11
    row_extract_end += 11
    col_extract_start = 0
    col_extract_end = 4

AAA 2024-06-20 00:00:00
AAA 2024-07-20 00:00:00
AAA 2024-08-20 00:00:00
AAA 2024-09-20 00:00:00
AAA 2024-06-20 00:00:00
AAA 2024-07-20 00:00:00
AAA 2024-08-20 00:00:00
AAA 2024-09-20 00:00:00
AAA 2024-06-20 00:00:00
AAA 2024-07-20 00:00:00
AAA 2024-08-20 00:00:00
AAA 2024-09-20 00:00:00
AAA 2024-06-20 00:00:00
AAA 2024-07-20 00:00:00
AAA 2024-08-20 00:00:00
AAA 2024-09-20 00:00:00
BBB 2024-06-20 00:00:00
BBB 2024-07-20 00:00:00
BBB 2024-08-20 00:00:00
BBB 2024-09-20 00:00:00
BBB 2024-06-20 00:00:00
BBB 2024-07-20 00:00:00
BBB 2024-08-20 00:00:00
BBB 2024-09-20 00:00:00
BBB 2024-06-20 00:00:00
BBB 2024-07-20 00:00:00
BBB 2024-08-20 00:00:00
BBB 2024-09-20 00:00:00
BBB 2024-06-20 00:00:00
BBB 2024-07-20 00:00:00
BBB 2024-08-20 00:00:00
BBB 2024-09-20 00:00:00
CCC 2024-06-20 00:00:00
CCC 2024-07-20 00:00:00
CCC 2024-08-20 00:00:00
CCC 2024-09-20 00:00:00
CCC 2024-06-20 00:00:00
CCC 2024-07-20 00:00:00
CCC 2024-08-20 00:00:00
CCC 2024-09-20 00:00:00
CCC 2024-06-20 00:00:00
CCC 2024-07-20 0

In [287]:
more_item

,ItemName,2024-06-20 00:00:00,2024-07-20 00:00:00,2024-08-20 00:00:00,2024-09-20 00:00:00,2024-10-20 00:00:00,2024-11-20 00:00:00,2024-12-20 00:00:00,2025-01-20 00:00:00,2025-02-20 00:00:00,2025-03-20 00:00:00,2025-04-20 00:00:00,2025-05-20 00:00:00
0,apple,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,banana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,orange,8.0,12.0,8.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,lemon,4.0,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,red,8.0,8.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,blue,8.0,8.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,yellow,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,green,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,kite,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,queen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [288]:
less_item

,ItemName,2024-06-20 00:00:00,2024-07-20 00:00:00,2024-08-20 00:00:00,2024-09-20 00:00:00,2024-10-20 00:00:00,2024-11-20 00:00:00,2024-12-20 00:00:00,2025-01-20 00:00:00,2025-02-20 00:00:00,2025-03-20 00:00:00,2025-04-20 00:00:00,2025-05-20 00:00:00
0,apple,12.0,12.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,banana,8.0,4.0,4.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,orange,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,lemon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,red,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,blue,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,yellow,4.0,12.0,8.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,green,8.0,0.0,4.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,kite,0.0,4.0,8.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,queen,0.0,0.0,8.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [289]:
more_item_value

,ItemName,2024-06-20 00:00:00,2024-07-20 00:00:00,2024-08-20 00:00:00,2024-09-20 00:00:00,2024-10-20 00:00:00,2024-11-20 00:00:00,2024-12-20 00:00:00,2025-01-20 00:00:00,2025-02-20 00:00:00,2025-03-20 00:00:00,2025-04-20 00:00:00,2025-05-20 00:00:00
0,apple,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,banana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,orange,129200.0,71600.0,80000.0,26800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,lemon,80000.0,136000.0,40000.0,3600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,red,43120.0,81600.0,200000.0,60000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,blue,10000.0,126000.0,200000.0,30800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,yellow,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,green,0.0,3576000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,kite,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,queen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [290]:
less_item_value

,ItemName,2024-06-20 00:00:00,2024-07-20 00:00:00,2024-08-20 00:00:00,2024-09-20 00:00:00,2024-10-20 00:00:00,2024-11-20 00:00:00,2024-12-20 00:00:00,2025-01-20 00:00:00,2025-02-20 00:00:00,2025-03-20 00:00:00,2025-04-20 00:00:00,2025-05-20 00:00:00
0,apple,-12000.0,-64000.0,-80000.0,-26800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,banana,76800.0,-56000.0,52000.0,126800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,orange,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,lemon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,red,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,blue,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,yellow,-22000.0,-74800.0,-64000.0,-140000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,green,-16000.0,0.0,-16000.0,-72000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,kite,0.0,1200.0,2400.0,1200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,queen,0.0,0.0,-72000.0,-64000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [291]:
from openpyxl.utils.dataframe import dataframe_to_rows

# Create worksheets and set their names
worksheet_names = ["WS1", "WS2", "WS3", "WS4"]
worksheets = [workbook.create_sheet(title=name) for name in worksheet_names]

# Get references to the existing worksheets
#worksheet_names = ["WS1", "WS2", "WS3", "WS4"]
#worksheets = [workbook[name] for name in worksheet_names]

# Define the data to be written to worksheets
data = [
    more_item,
    less_item,
    more_item_value,
    less_item_value
]

# Write data to worksheets
for sheet, value in zip(worksheets, data):
    column_names = value.columns.tolist()
    sheet.append(column_names)  # Write column names in the first row
    for row in dataframe_to_rows(value, index=False, header=False):
        sheet.append(row)

# Save the workbook
workbook.save("Temp.xlsx")